# House Prices: Predicting Sale Price with XGBoost

1. Load and explore the **House Prices** dataset (Ames Housing).  
2. Do some basic data cleaning and feature engineering.  
3. Train and compare three prediction models:

   - **Linear Regression**
   - **Random Forest Regressor** 
   - **XGBoost Regressor**

4. Evaluate the models using **R²** and **RMSE**.  


In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import joblib
import xgboost
from xgboost import XGBRegressor

data_path = "housing.csv"
df = pd.read_csv(data_path)

df.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD         Normal     208500  
1   2007        WD         Normal     181500  
2   2008        WD         Normal     223500  
3   2006        WD        Abnorml     140000  
4   2008        WD         Normal     250000  

[5 rows x 81 columns]

In [2]:
print("Shape of data:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nInfo:")
df.info()

Shape of data: (1460, 81)

Columns:
['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',

In [3]:
# Target variable
target_col = "SalePrice"

feature_cols = [
    "OverallQual",   # Overall material and finish quality (1-10)
    "GrLivArea",     # Above ground living area (square feet)
    "GarageCars",    # Size of garage in car capacity
    "TotalBsmtSF",   # Total basement area (square feet)
    "FullBath",      # Full bathrooms above grade
    "YearBuilt",     # Original construction date
    "Neighborhood"   # Physical locations within Ames city limits
]

X = df[feature_cols].copy()
y = df[target_col].copy()

print("Features shape:", X.shape)
print("Target shape:", y.shape)

# Show summary stats for numeric features
print("\nSummary statistics (numeric features):")
X.describe().T


Features shape: (1460, 7)
Target shape: (1460,)

Summary statistics (numeric features):


count         mean         std     min      25%     50%  \
OverallQual  1460.0     6.099315    1.382997     1.0     5.00     6.0   
GrLivArea    1460.0  1515.463699  525.480383   334.0  1129.50  1464.0   
GarageCars   1460.0     1.767123    0.747315     0.0     1.00     2.0   
TotalBsmtSF  1460.0  1057.429452  438.705324     0.0   795.75   991.5   
FullBath     1460.0     1.565068    0.550916     0.0     1.00     2.0   
YearBuilt    1460.0  1971.267808   30.202904  1872.0  1954.00  1973.0   

                 75%     max  
OverallQual     7.00    10.0  
GrLivArea    1776.75  5642.0  
GarageCars      2.00     4.0  
TotalBsmtSF  1298.25  6110.0  
FullBath        2.00     3.0  
YearBuilt    2000.00  2010.0

In [4]:
missing_ratio = X.isna().mean().sort_values(ascending=False)
print("Missing value ratio per selected feature:")
print(missing_ratio)

Missing value ratio per selected feature:
OverallQual     0.0
GrLivArea       0.0
GarageCars      0.0
TotalBsmtSF     0.0
FullBath        0.0
YearBuilt       0.0
Neighborhood    0.0
dtype: float64


In [5]:
# Identify numeric and categorical columns

numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

print("Numeric features:", numeric_features)
print("Categorical features:", categorical_features)

Numeric features: ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
Categorical features: ['Neighborhood']


In [6]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# - Fill missing values with the most frequent value
# - One-hot encode them
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (1168, 7)
Test set shape: (292, 7)


In [23]:
def evaluate_regression_model(name, model, X_train, y_train, X_test, y_test):
    """
    Fit the model, make predictions, and print R² and RMSE.
    Returns the fitted model and the metrics.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)


    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)                       

    print(f"Model: {name}")
    print(f"  R²   : {r2:.4f}")
    print(f"  RMSE : {rmse:,.2f}")
    print("-" * 40)

    return model, r2, rmse

In [9]:
# Model 1 - Linear Regression

linreg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])

linreg_model, r2_lin, rmse_lin = evaluate_regression_model(
    "Linear Regression",
    linreg_pipeline,
    X_train,
    y_train,
    X_test,
    y_test
)

Model: Linear Regression
  R²   : 0.8267
  RMSE : 36,455.29
----------------------------------------


In [10]:
# Model 2 - Random Forest Regressor

rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ))
])

rf_model, r2_rf, rmse_rf = evaluate_regression_model(
    "Random Forest Regressor",
    rf_pipeline,
    X_train,
    y_train,
    X_test,
    y_test
)

Model: Random Forest Regressor
  R²   : 0.8956
  RMSE : 28,294.48
----------------------------------------


In [11]:
# Model 3 - XGBoost Regressor 

xgb_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        objective="reg:squarederror",
        n_jobs=-1
    ))
])

xgb_model, r2_xgb, rmse_xgb = evaluate_regression_model(
    "XGBoost Regressor",
    xgb_pipeline,
    X_train,
    y_train,
    X_test,
    y_test
)

Model: XGBoost Regressor
  R²   : 0.9082
  RMSE : 26,536.61
----------------------------------------


In [12]:
# Compare models

results = pd.DataFrame({
    "Model": ["Linear Regression", "Random Forest", "XGBoost"],
    "R2": [r2_lin, r2_rf, r2_xgb],
    "RMSE": [rmse_lin, rmse_rf, rmse_xgb]
})

results

Model        R2          RMSE
0  Linear Regression  0.826736  36455.285528
1      Random Forest  0.895627  28294.481217
2            XGBoost  0.908193  26536.610183

In [13]:
metrics = {
    "Linear Regression": r2_lin,
    "Random Forest": r2_rf,
    "XGBoost": r2_xgb
}

best_model_name = max(metrics, key=metrics.get)
print("Best model based on R²:", best_model_name)

if best_model_name == "Linear Regression":
    best_model = linreg_model
elif best_model_name == "Random Forest":
    best_model = rf_model
else:
    best_model = xgb_model

# Save the entire pipeline (preprocessing + model)
model_filename = "best_house_price_model.pkl"
joblib.dump(best_model, model_filename)

print(f"Saved best model to {model_filename}")

Best model based on R²: XGBoost
Saved best model to best_house_price_model.pkl


In [14]:
loaded_model = joblib.load(model_filename)


example_features = X_test.iloc[[0]]  # keep as DataFrame

true_price = y_test.iloc[0]

# Make a prediction
predicted_price = loaded_model.predict(example_features)[0]

print("Example house features:")
display(example_features)

print(f"\nTrue SalePrice     : {true_price:,.0f}")
print(f"Predicted SalePrice: {predicted_price:,.0f}")

Example house features:


OverallQual  GrLivArea  GarageCars  TotalBsmtSF  FullBath  YearBuilt  \
892            6       1068           1         1059         1       1963   

    Neighborhood  
892       Sawyer


True SalePrice     : 154,500
Predicted SalePrice: 133,869


In [15]:
import joblib
from pathlib import Path

best_model_path = Path("house_price_api") / "best_house_price_model.pkl"
joblib.dump(best_model, best_model_path)

print("Saved model to:", best_model_path)

Saved model to: house_price_api\best_house_price_model.pkl


In [16]:
X_test.iloc[0]

OverallQual          6
GrLivArea         1068
GarageCars           1
TotalBsmtSF       1059
FullBath             1
YearBuilt         1963
Neighborhood    Sawyer
Name: 892, dtype: object

In [17]:
y_test.iloc[0]

np.int64(154500)

In [20]:
example = X_test.iloc[0]
true_price = y_test.iloc[0]

print(example)
print("True SalePrice:", true_price)

OverallQual          6
GrLivArea         1068
GarageCars           1
TotalBsmtSF       1059
FullBath             1
YearBuilt         1963
Neighborhood    Sawyer
Name: 892, dtype: object
True SalePrice: 154500


In [22]:
import requests

url = "http://127.0.0.1:5000/api/predict"

payload = {
    "OverallQual": int(example["OverallQual"]),
    "GrLivArea": float(example["GrLivArea"]),
    "GarageCars": int(example["GarageCars"]),
    "TotalBsmtSF": float(example["TotalBsmtSF"]),
    "FullBath": int(example["FullBath"]),
    "YearBuilt": int(example["YearBuilt"]),
    "Neighborhood": str(example["Neighborhood"])
}

response = requests.post(url, json=payload)

print("Status code:", response.status_code)
print("Response JSON:", response.json())
print("True SalePrice:", true_price)

Status code: 200
Response JSON: {'prediction': 133869.25}
True SalePrice: 154500


## Summary

- I used the House Prices (Ames Housing) dataset and predicted the SalePrice of homes.
- I selected a small but meaningful set of features:
OverallQual, GrLivArea, GarageCars, TotalBsmtSF, FullBath, YearBuilt, and Neighborhood.
- I built a preprocessing pipeline that:
Imputes missing numeric values with the median and scales them.
Imputes missing categorical values with the most frequent category and one-hot encodes them.
- I trained and compared three models:
Linear Regression,
Random Forest Regressor,
XGBoost Regressor
- I evaluated the models using R² and RMSE and selected the best one based on R².

